In [13]:
import pandas as pd
def print_ratio(df,x_values,save_name):
    # 计算每个组合的平均return
    avg_return = df.groupby(['algorithm', 'train mode', 'test mode'])['ep ret'].mean().reset_index()

    result_df = pd.DataFrame(columns=['algorithm'])
    algorithms = avg_return['algorithm'].unique()

    for algo in algorithms:
        algo_data = avg_return[avg_return['algorithm'] == algo]
        
        algo_ratios = {'algorithm': algo}
        for x in x_values:
            before_condition = (avg_return['train mode'] == x - 1) & (avg_return['test mode'] == x - 1)
            max_ret = avg_return[before_condition]['ep ret'].max()
            
            # condition after training on new task
            after_condition = (algo_data['train mode'] == x) & (algo_data['test mode'] == x - 1)
            after_resutls = algo_data[after_condition]
            
            # condition before training on new task
            before_condition = (algo_data['train mode'] == x - 1) & (algo_data['test mode'] == x - 1)
            before_results = algo_data[before_condition]
            
            if not after_resutls.empty and not before_results.empty:
                performance_before = before_results['ep ret'].values[0] / max_ret
                performance_after = after_resutls['ep ret'].values[0] / max_ret
                # ratio = (performance_before - performance_after)
                ratio = (performance_before - performance_after) / performance_before
                if algo in ['MaskNet', "PackNet"]:
                    print(performance_before, performance_after)
                    print(ratio,ratio / performance_before)
                # ratio = (performance_before - performance_after) 
                algo_ratios[f'forgetting on {x}'] = ratio 
            else:
                algo_ratios[f'forgetting on {x}'] = None  
        
        # algo_ratios_df = pd.DataFrame(algo_ratios)
        # result_df = pd.concat([result_df, algo_ratios_df], ignore_index=True)
        result_df = result_df.append(algo_ratios, ignore_index=True)

    # print(result_df)
    numeric_df = result_df.select_dtypes(include='number')

    result_df['mean'] = numeric_df.mean(axis=1)
    # print(resul)
    result_df.to_csv(f'{save_name}.csv', index=False)
    print(result_df)
    
def print_final_performance(df,x_values,save_name):
    # 计算每个组合的平均return
    avg_return = df.groupby(['algorithm', 'train mode', 'test mode'])['ep ret'].mean().reset_index()

    result_df = pd.DataFrame(columns=['algorithm'])
    algorithms = avg_return['algorithm'].unique()
    train_mode = x_values[-1]
    for algo in algorithms:
        algo_data = avg_return[avg_return['algorithm'] == algo]
        
        algo_ratios = {'algorithm': algo}
        # 遍历每个x值
        for x in x_values:
            cur_condition = (avg_return['train mode'] == x) & (avg_return['test mode'] == x)
            max_ret = avg_return[cur_condition]['ep ret'].max()
            
            # 筛选出train mode = x, test mode = x-1的行
            condition1 = (algo_data['train mode'] == train_mode) & (algo_data['test mode'] == x)
            result1 = algo_data[condition1]
            
            # 筛选出train mode = x-1, test mode = x-1的行
            prev_condition = (algo_data['train mode'] == x) & (algo_data['test mode'] == x)
            result2 = algo_data[prev_condition]
            
            # 如果两组数据都存在，则计算比值
            if not result1.empty and not result2.empty:
                ratio = result1['ep ret'].values[0] / max_ret
                algo_ratios[f'{x}'] = ratio  # 动态列名
            else:
                algo_ratios[f'{x}'] = 0  # 如果没有匹配的数据，则设为None
        
        result_df = result_df.append(algo_ratios, ignore_index=True)

    # print(result_df)
    numeric_df = result_df.select_dtypes(include='number')

    result_df['mean'] = numeric_df.mean(axis=1)
    # print(resul)
    result_df.to_csv(f'{save_name}.csv', index=False)
    print(result_df)

In [14]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('eval_Freeway/eval_results.csv')
x_values = range(1,8)
print_ratio(df, x_values, 'eval_Freeway/forgetting_results')

# 读取CSV文件
df = pd.read_csv('eval_SpaceInvaders/eval_results.csv')
x_values = range(1,10)
print_ratio(df, x_values, 'eval_SpaceInvaders/forgetting_results')

/tmp/ipykernel_119163/2137582870.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:40: FutureWarning: T

0.9570815450643777 0.9742489270386265
-0.017937219730941603 -0.018741579360131808
0.167420814479638 0.13574660633484162
0.1891891891891892 1.1300219138056977
0.35398230088495575 0.12831858407079644
0.6375000000000001 1.8009375000000003
0.424 0.22399999999999998
0.47169811320754723 1.1124955500178002
0.6759259259259259 0.7037037037037037
-0.04109589041095892 -0.060799399512103613
0.3436123348017621 0.2290748898678414
0.3333333333333333 0.97008547008547
0.26200873362445415 0.2183406113537118
0.16666666666666663 0.636111111111111
0.9613733905579398 0.9570815450643777
0.004464285714285574 0.004643654336734549
0.13574660633484162 0.13574660633484162
0.0 0.0
0.07522123893805309 0.0663716814159292
0.11764705882352942 1.5640138408304503
0.36 0.08
0.7777777777777777 2.1604938271604937
0.6759259259259259 0.6450617283950617
0.045662100456621044 0.06755488834678182
0.3392070484581498 0.3480176211453745
-0.02597402597402601 -0.07657277787147927
0.24017467248908297 0.27947598253275113
-0.16363636363

/tmp/ipykernel_119163/2137582870.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:40: FutureWarning: T

0.23241852487135506 0.31046312178387653
-0.33579335793357945 -1.4447788020315633
0.27815063385533184 0.20507084265473527
0.26273458445040215 0.944576615946352
0.200139470013947 0.26778242677824265
-0.337979094076655 -1.6887178428777814
0.22078720787207873 0.1931119311193112
0.12534818941504183 0.5677330250386017
0.12585499316005472 0.07729138166894665
0.38586956521739124 3.0659853497164455
0.0925700365408039 0.06455542021924482
0.3026315789473685 3.2692174515235464
0.12804878048780488 0.08384146341463415
0.34523809523809523 2.696145124716553
0.10067114093959731 0.07181208053691275
0.2866666666666666 2.847555555555555
0.24432166973603436 0.21240024554941683
0.1306532663316582 0.5347592232519377
0.5291595197255574 0.4099485420240137
0.22528363047001618 0.4257385950211327
0.33184190902311705 0.2371364653243848
0.2853932584269662 0.8600277742709252
0.2524407252440725 0.2280334728033473
0.0966850828729281 0.3830011293916544
0.21525215252152521 0.22263222632226323
-0.034285714285714336 -0.15

In [17]:
import pandas as pd

# 读取CSV文件
# df = pd.read_csv('eval_Freeway/eval_results.csv')
# for x in range(1,9):
#     x_values = range(x)
#     print_final_performance(df, x_values, f'eval_Freeway/{x}_final_perf_results')

df = pd.read_csv('eval_SpaceInvaders/eval_results.csv')
for x in range(1,11):
    x_values = range(x)
    print_final_performance(df, x_values, f'eval_SpaceInvaders/{x}_final_perf_results')

/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: T

  algorithm         0      mean
0  CompoNet  0.849914  0.849914
1        F1  0.987993  0.987993
2        FN  1.000000  1.000000
3   FuseNet  0.803602  0.803602
4   MaskNet  0.232419  0.232419
5   PackNet  0.529160  0.529160
6   ProgNet  0.823328  0.823328
  algorithm         0         1      mean
0  CompoNet  0.666381  0.747204  0.706792
1        F1  0.845626  0.890380  0.868003
2        FN  0.955403  0.993289  0.974346
3   FuseNet  0.819039  1.000000  0.909520
4   MaskNet  0.310463  0.278151  0.294307
5   PackNet  0.409949  0.331842  0.370895
6   ProgNet  0.305317  0.746458  0.525888


/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: T

  algorithm         0         1         2      mean
0  CompoNet  0.850772  0.437733  1.000000  0.762835
1        F1  0.520583  0.459359  0.630404  0.536782
2        FN  0.367067  0.413125  0.896792  0.558995
3   FuseNet  0.673242  0.302013  0.673640  0.549632
4   MaskNet  0.379074  0.205071  0.200139  0.261428
5   PackNet  0.451973  0.237136  0.252441  0.313850
6   ProgNet  0.795026  0.331096  0.711994  0.612705
  algorithm         0         1         2         3      mean
0  CompoNet  0.498285  0.548844  0.887029  1.000000  0.733540
1        F1  0.674957  0.753169  0.617155  0.677737  0.680754
2        FN  0.433962  0.649515  1.129707  0.808733  0.755479
3   FuseNet  0.588336  0.680835  0.831241  0.730627  0.707760
4   MaskNet  0.262436  0.211037  0.267782  0.220787  0.240510
5   PackNet  0.467410  0.346010  0.228033  0.215252  0.314177
6   ProgNet  0.626930  0.536167  0.600418  0.572571  0.584021
  algorithm         0         1         2         3         4      mean
0  CompoNet  0.5

/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: T

  algorithm         0         1         2         3         4         5  \
0  CompoNet  0.732419  0.426547  1.078103  0.921279  1.270862  0.815469   
1        F1  0.555746  0.752424  0.373780  0.487085  0.313953  0.289281   
2        FN  0.730703  0.445936  0.900976  0.673432  1.071819  0.838611   
3   FuseNet  0.524014  0.550336  0.903068  0.795203  1.090287  0.809988   
4   MaskNet  0.431389  0.322893  0.303347  0.276138  0.120383  0.070646   
5   PackNet  0.422813  0.293811  0.317992  0.261993  0.143639  0.069428   
6   ProgNet  0.488851  0.648770  0.494421  0.470480  0.080027  0.186967   

          6         7      mean  
0  0.794715  0.883893  0.865411  
1  0.134146  0.341611  0.406003  
2  0.838415  1.000000  0.812487  
3  0.761687  0.891275  0.790732  
4  0.083841  0.100671  0.213664  
5  0.091463  0.063758  0.208112  
6  0.079776  0.430872  0.360021  
  algorithm         0         1         2         3         4         5  \
0  CompoNet  0.666381  0.551827  0.920502  0.818573 

/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
/tmp/ipykernel_119163/2137582870.py:81: FutureWarning: T